# Creating & Analyzing Equity Portfolios (with rebalancing)

## Getting started

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("seaborn-v0_8")

In [ ]:
prices = pd.read_csv("stocks.csv", index_col = "Date", parse_dates = ["Date"])
prices

In [ ]:
returns = prices.pct_change().dropna() 
returns

In [ ]:
returns.info()

In [ ]:
def ann_risk_return(returns_df): # assumes simple returns as input
    summary = pd.DataFrame(index = returns_df.columns)
    summary["ann. Risk"] = returns_df.std() * np.sqrt(252)
    log_returns = np.log(returns_df + 1)
    summary["CAGR"] = np.exp(log_returns.mean() * 252) - 1
    return summary

In [ ]:
summary = ann_risk_return(returns.iloc[1:])
summary

## Creating Random Portfolios (Part 1)

In [ ]:
noa = len(returns.columns) # number of assets
noa

In [ ]:
nop = 10000 # number of random portfolios
nop

In [ ]:
# 80,000 random floats between 0 and 1
np.random.seed(123)
matrix = np.random.random(noa * nop).reshape(nop, noa)

In [ ]:
matrix

In [ ]:
matrix.shape

In [ ]:
matrix.sum(axis = 1, keepdims= True)

In [ ]:
weights = matrix / matrix.sum(axis = 1, keepdims= True) # make portfolio weights summing up to 1
weights

In [ ]:
weights.sum(axis = 1, keepdims= True)

__one Portfolio__

In [ ]:
w1 = weights[0]
w1

__Daily rebalancing of weights.__

In [ ]:
returns.mul(w1).sum(axis = 1) # weighted average simple returns over time

In [ ]:
port1 = returns.dot(w1) # dot product
port1

-> __Active rebalancing__! __Weights restored__ to initial/target weights at the end of each day. <br>
-> selling daily winners and buying daily losers. (__Contrarian Trading Strategy__)


In [ ]:
ann_risk_return(port1.to_frame())

## Creating Random Portfolios (Part 2)

__many Portfolios__

In [ ]:
weights

In [ ]:
port_ret= returns.dot(weights.T)
port_ret # weighted average simple returns for 10,000 portfolios

In [ ]:
port_summary = ann_risk_return(port_ret)
port_summary

In [ ]:
summary

In [ ]:
plt.figure(figsize = (15, 9))
plt.scatter(port_summary.loc[:, "ann. Risk"], port_summary.loc[:, "CAGR"],s= 20, color = "red")
plt.scatter(summary.loc[:, "ann. Risk"], summary.loc[:, "CAGR"], s= 50, color = "black", marker = "D")
for i in summary.index:
    plt.annotate(i, xy=(summary.loc[i, "ann. Risk"]+0.01, summary.loc[i, "CAGR"]+0.01), size = 15)
plt.xlabel("ann. Risk (std)", fontsize = 15)
plt.ylabel("CAGR", fontsize = 15)
plt.title("Risk/Return", fontsize = 20)
plt.show()

## Performance Measurement: Risk-adjusted Return

__Risk-adjusted-Return (RaR): CAGR per unit of Risk. (similar to Sharpe Ratio)__

In [ ]:
summary["RaR"] = summary["CAGR"].div(summary["ann. Risk"])
summary

In [ ]:
port_summary["RaR"] = port_summary["CAGR"].div(port_summary["ann. Risk"])

In [ ]:
port_summary.sort_values("RaR")

In [ ]:
vmin = port_summary.RaR.min()
vmin

In [ ]:
vmax = port_summary.RaR.max()
vmax

In [ ]:
plt.figure(figsize = (15, 8))
plt.scatter(port_summary.loc[:, "ann. Risk"], port_summary.loc[:, "CAGR"], s= 20, 
            c = port_summary.loc[:, "RaR"], cmap = "coolwarm", vmin = vmin, vmax = vmax, alpha = 0.66)
plt.colorbar()
plt.scatter(summary.loc[:, "ann. Risk"], summary.loc[:, "CAGR"],s= 50, marker = "D", c = "black")
for i in summary.index:
    plt.annotate(i, xy=(summary.loc[i, "ann. Risk"]+0.01, summary.loc[i, "CAGR"]+0.01), size = 15)
plt.xlabel("ann. Risk (std)", fontsize = 15)
plt.ylabel("CAGR", fontsize = 15)
plt.title("Risk-adjusted Return", fontsize = 20)
plt.show()

## Portfolio Optimization

__Note: We are optimizing the past here (backward looking)!__

In [ ]:
import scipy.optimize as sco
pd.options.display.float_format = '{:.4f}'.format
np.set_printoptions(suppress = True)

In [ ]:
returns 

In [ ]:
#calculate portfolio CAGR (based on weights)
def port_ret(weights):
    simple_ret = returns.dot(weights.T)
    log_returns = np.log(simple_ret + 1)
    cagr = np.exp(log_returns.mean() * 252) - 1
    return cagr

In [ ]:
#calculate annualized portfolio volatility (based on weights)
def port_vol(weights):
    simple_ret = returns.dot(weights.T)
    return simple_ret.std() * np.sqrt(252)

In [ ]:
#define function to be minimized (sco only supports minimize, not maximize)
#-> maximize RaR == minimize RaR * (-1)
def min_func_RaR(weights): 
     return -(port_ret(weights)) / port_vol(weights) #Risk-adjusted Return * (-1)

In [ ]:
#number of assets
noa = len(returns.columns)
noa

In [ ]:
#equal weights (starting point of optimization)
eweigths = np.full(noa, 1/noa)
eweigths

In [ ]:
#constraint: weights must sum up to 1 -> sum of weights - 1 = 0
cons = ({"type": "eq", "fun": lambda x: np.sum(x) - 1})

In [ ]:
#bounds: all weights shall be between 0 and 1 -> can be changed
bnds =  tuple((0, 1) for x in range(noa))
bnds

In [ ]:
#run optimization based on function to be minimized, starting with equal weights and based on respective bounds and constraints
opts = sco.minimize(min_func_RaR, eweigths, method = "SLSQP", bounds = bnds, constraints= cons)

In [ ]:
#output of optimization
opts

In [ ]:
#getting the optimal weights
optimal_weights = opts["x"]
optimal_weights

In [ ]:
pd.Series(data = optimal_weights, index = returns.columns).sort_values(ascending = False).head(20)

-> __Optimization (without bounds) does not necessarily lead to practical/factual diversification.__

In [ ]:
#cagr of the optimal portfolio
cagr_opt = port_ret(optimal_weights)
cagr_opt

In [ ]:
#volatility of the optimal portfolio
vol_opt = port_vol(optimal_weights)
vol_opt

In [ ]:
#RaR of the optimal portfolio
RaR_opt = -min_func_RaR(optimal_weights)
RaR_opt

In [ ]:
plt.figure(figsize = (15, 8))
plt.scatter(port_summary.loc[:, "ann. Risk"], port_summary.loc[:, "CAGR"], s= 20, 
            c = port_summary.loc[:, "RaR"], cmap = "coolwarm", vmin = vmin, vmax = vmax, alpha = 0.66)
plt.colorbar()
plt.scatter(summary.loc[:, "ann. Risk"], summary.loc[:, "CAGR"],s= 50, marker = "D", c = "black")
for i in summary.index:
    plt.annotate(i, xy=(summary.loc[i, "ann. Risk"]+0.01, summary.loc[i, "CAGR"]+0.01), size = 15)
plt.scatter(x = vol_opt, y = cagr_opt, s = 100, marker = "X", c = "purple") # best Portfolio
plt.xlabel("ann. Risk (std)", fontsize = 15)
plt.ylabel("CAGR", fontsize = 15)
plt.title("The optimal Portfolio", fontsize = 20)
plt.show()

__Reminder: We are optimizing the past here (backward looking).__ <br>
-> Very unlikely we had selected this optimal portfolio back in 2017 (__look ahead bias__)! <br>
-> Very unlikely this will be the optimal portfolio in the future (__past performance is not a good indicator for future performance__)!

## Minimum Variance Portfolio

(use code above with following replacements:)

In [ ]:
# Minimum Variance Portfolio
opts = sco.minimize(port_vol, eweigths, method = "SLSQP", bounds = bnds, constraints= cons)

## Maximum Return Portfolio

(use code above with following replacements:)

In [ ]:
#calculate portfolio CAGR (based on weights)
def port_ret(weights):
    simple_ret = returns.dot(weights.T)
    log_returns = np.log(simple_ret + 1)
    cagr = np.exp(log_returns.mean() * 252) - 1
    return cagr * (-1)

In [ ]:
# Maximum Return Portfolio
opts = sco.minimize(port_ret, eweigths, method = "SLSQP", bounds = bnds, constraints= cons)

In [ ]:
#cagr of the optimal portfolio
cagr_opt = -port_ret(optimal_weights)
cagr_opt

## The Efficient Frontier

__Idea: Find for each return level the portfolio with the lowest volatility.__

In [ ]:
returns

In [ ]:
summary

In [ ]:
#calculate portfolio CAGR (based on weights)
def port_ret(weights):
    simple_ret = returns.dot(weights.T).iloc[1:]
    log_returns = np.log(simple_ret + 1)
    cagr = np.exp(log_returns.mean() * 252) - 1
    return cagr

In [ ]:
#calculate annualized portfolio volatility (based on weights)
def port_vol(weights):
    simple_ret = returns.dot(weights.T).iloc[1:]
    return simple_ret.std() * np.sqrt(252)

In [ ]:
noa = len(returns.columns)
noa

In [ ]:
#equal weights (starting point of optimization)
eweigths = np.full(noa, 1/noa)
eweigths

In [ ]:
tcagrs = np.linspace(summary.CAGR.min(), summary.CAGR.max(), 100)
tcagrs # evenly spaced target returns between min and max

In [ ]:
# portfolio return == tcagr
cons = ({"type": "eq", "fun": lambda x: port_ret(x) - tcagr},
       {"type": "eq", "fun": lambda x: np.sum(x) - 1})

In [ ]:
bnds =  tuple((0, 1) for x in range(noa))
bnds

In [ ]:
vols = []

In [ ]:
#for each target return, find the portfolio with the lowest volatility
for tcagr in tcagrs:
    res = sco.minimize(port_vol, eweigths, method = "SLSQP", bounds = bnds, constraints = cons)
    vols.append(res["fun"])
vols = np.array(vols)

In [ ]:
vols

In [ ]:
plt.figure(figsize = (15, 8))
plt.scatter(port_summary.loc[:, "ann. Risk"], port_summary.loc[:, "CAGR"], s= 20, 
            c = port_summary.loc[:, "RaR"], cmap = "coolwarm", vmin = vmin, vmax = vmax, alpha = 0.66)
plt.colorbar()
plt.scatter(summary.loc[:, "ann. Risk"], summary.loc[:, "CAGR"],s= 50, marker = "D", c = "black")
for i in summary.index:
    plt.annotate(i, xy=(summary.loc[i, "ann. Risk"]+0.01, summary.loc[i, "CAGR"]+0.01), size = 15)
plt.plot(vols, tcagrs) # efficient frontier
plt.xlabel("ann. Risk (std)", fontsize = 15)
plt.ylabel("CAGR", fontsize = 15)
plt.title("The Efficient Frontier", fontsize = 20)
plt.show()

## Comparison: daily Rebalancing vs. no Rebalancing

__Efficient Frontier without rebalancing:__

In [ ]:
cagr_no = np.array([-0.21321957, -0.20504081, -0.19686206, -0.18868331, -0.18050455,
       -0.1723258 , -0.16414704, -0.15596829, -0.14778953, -0.13961078,
       -0.13143202, -0.12325327, -0.11507451, -0.10689576, -0.098717  ,
       -0.09053825, -0.08235949, -0.07418074, -0.06600198, -0.05782323,
       -0.04964447, -0.04146572, -0.03328696, -0.02510821, -0.01692945,
       -0.0087507 , -0.00057194,  0.00760681,  0.01578557,  0.02396432,
        0.03214308,  0.04032183,  0.04850058,  0.05667934,  0.06485809,
        0.07303685,  0.0812156 ,  0.08939436,  0.09757311,  0.10575187,
        0.11393062,  0.12210938,  0.13028813,  0.13846689,  0.14664564,
        0.1548244 ,  0.16300315,  0.17118191,  0.17936066,  0.18753942,
        0.19571817,  0.20389693,  0.21207568,  0.22025444,  0.22843319,
        0.23661195,  0.2447907 ,  0.25296946,  0.26114821,  0.26932697,
        0.27750572,  0.28568448,  0.29386323,  0.30204198,  0.31022074,
        0.31839949,  0.32657825,  0.334757  ,  0.34293576,  0.35111451,
        0.35929327,  0.36747202,  0.37565078,  0.38382953,  0.39200829,
        0.40018704,  0.4083658 ,  0.41654455,  0.42472331,  0.43290206,
        0.44108082,  0.44925957,  0.45743833,  0.46561708,  0.47379584,
        0.48197459,  0.49015335,  0.4983321 ,  0.50651086,  0.51468961,
        0.52286837,  0.53104712,  0.53922587,  0.54740463,  0.55558338,
        0.56376214,  0.57194089,  0.58011965,  0.5882984 ,  0.59647716])

In [ ]:
vols_no = np.array([0.43230733, 0.4082562 , 0.38776821, 0.36971057, 0.35298005,
       0.33746244, 0.32307554, 0.30974812, 0.29741952, 0.28603623,
       0.27555009, 0.26591947, 0.25710661, 0.2490747 , 0.24178985,
       0.23521878, 0.22932929, 0.22408966, 0.2194684 , 0.21543409,
       0.2119554 , 0.20898808, 0.2064256 , 0.20423296, 0.20239196,
       0.20088063, 0.19967245, 0.19867169, 0.19785033, 0.19719909,
       0.19670841, 0.19636225, 0.19615204, 0.19607055, 0.19611186,
       0.19626816, 0.19653368, 0.19690195, 0.19736735, 0.19792419,
       0.19867678, 0.19968965, 0.20093869, 0.20241949, 0.20410759,
       0.20598642, 0.20804255, 0.21026158, 0.2126309 , 0.21513937,
       0.21779366, 0.22060839, 0.22356945, 0.22680792, 0.23038234,
       0.23426203, 0.23842321, 0.24284222, 0.2474965 , 0.25236585,
       0.25743184, 0.2626779 , 0.26808938, 0.273651  , 0.27935259,
       0.28518299, 0.29113258, 0.29719306, 0.30335721, 0.30961907,
       0.31597335, 0.32252773, 0.32942379, 0.33662063, 0.34408425,
       0.3517883 , 0.35971312, 0.36784493, 0.37617501, 0.3846992 ,
       0.39341798, 0.40233485, 0.41145694, 0.42079451, 0.4303609 ,
       0.44017233, 0.45024792, 0.46060983, 0.47132985, 0.48248702,
       0.4941349 , 0.50633574, 0.51916219, 0.53269957, 0.54704879,
       0.56233037, 0.57869002, 0.59630658, 0.61540303, 0.63626358])

In [ ]:
plt.figure(figsize = (15, 8))
plt.scatter(port_summary.loc[:, "ann. Risk"], port_summary.loc[:, "CAGR"], s= 20, 
            c = port_summary.loc[:, "RaR"], cmap = "coolwarm", vmin = vmin, vmax = vmax, alpha = 0.66)
plt.colorbar()
plt.scatter(summary.loc[:, "ann. Risk"], summary.loc[:, "CAGR"],s= 50, marker = "D", c = "black")
for i in summary.index:
    plt.annotate(i, xy=(summary.loc[i, "ann. Risk"]+0.01, summary.loc[i, "CAGR"]+0.01), size = 15)
plt.plot(vols, tcagrs, label = "EF daily rebalancing") # efficient frontier with rebal.
plt.plot(vols_no, cagr_no, label = "EF no rebalancing") # efficient frontier without rebal.
plt.xlabel("ann. Risk (std)", fontsize = 15)
plt.ylabel("CAGR", fontsize = 15)
plt.legend(fontsize = 12)
plt.title("The Effect of Rebalancing", fontsize = 20)
plt.show()

__Upward Shift__ of Efficient Frontier:
- equal risk & more return or
- equal return & less risk

Two general effects of rebalancing: <br>
- maintain __higher degree of diversification__ / restoring weights (always positive)
- __contrarian trading__ / sell winners buy losers (can be positive or negative)

## Approximation of Rebalancing Costs

Can we really benefit from (daily) Rebalancing? <br>
Yes, but only if:
- __small changes__ in portfolio weights are possible (large portfolio or fractional trading)
- __Rebalancing Costs are lower__ than Rebalancing Benefits

In [ ]:
w1 # target weights to be reinstated at the end of each day 

In [ ]:
returns

In [ ]:
matrix = returns.add(1).mul(w1).values
matrix

In [ ]:
weights = matrix / matrix.sum(axis = 1, keepdims= True)
weights # weights at the end of each day (before rebalancing)

In [ ]:
weights.shape

In [ ]:
weights.sum(axis = 1)

In [ ]:
delta_weights = weights - w1
delta_weights

In [ ]:
np.abs(delta_weights)

In [ ]:
df = pd.DataFrame(np.abs(delta_weights))
df # deviation of weights

In [ ]:
daily_rebal = df.sum(axis = 1).mean()
daily_rebal

__-> on average, approx. 1.5% of total portfolio needs to be rebalanced every day (fractional trading).__

In [ ]:
ptc = 0.01 # worst case for US Stocks
ptc

In [ ]:
daily_ptc = daily_rebal * ptc
daily_ptc

In [ ]:
port1

In [ ]:
port1_ac = port1 - daily_ptc
port1_ac

In [ ]:
ann_risk_return(port1.to_frame())

In [ ]:
ann_risk_return(port1_ac.to_frame())

-> __Significant Rebalancing Costs__ eat up Rebalancing Benefits <br>
-> Solution: Reduce Rebalacing __Frequency to monthly/quarterly__ (reduces costs but also benefits)